In [1]:
import os
import sys

In [2]:
sys.path

['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/svu/e0384946/.ipython']

In [3]:
sys.path.append("/home/svu/e0384946/anaconda3/lib/python3.7/site-packages")

In [4]:
cwd = os.getcwd()
if "Pedestrian-Attribute-Recognition" in cwd:
    print("We're currently in Pedestrian-Attribute-Recognition folder")
else:
    print("Moving into Pedestrian-Attribute-Recognition folder")
    %cd Pedestrian-Attribute-Recognition

Moving into Pedestrian-Attribute-Recognition folder
/home/svu/e0384946/Pedestrian-Attribute-Recognition


In [5]:
import os
import pprint
from collections import OrderedDict, defaultdict

import numpy as np
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

from batch_engine import valid_trainer, batch_trainer
from config import argument_parser
from dataset.AttrDataset import AttrDataset, get_transform, AttrDataset_new, parse_transformation_dict
from loss.CE_loss import CEL_Sigmoid
from models.base_block import FeatClassifier, BaseClassifier
from models.resnet import resnet50
from tools.function import get_model_log_path, get_pedestrian_metrics
from tools.utils import time_str, save_ckpt, ReDirectSTD, set_seed
from torch.utils.tensorboard import SummaryWriter

import getpass
import inspect

set_seed(605)

/home/svu/e0384946/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/svu/e0384946/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/svu/e0384946/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/svu/e0384946/anaconda3/lib/python3.7/

In [6]:
writer = SummaryWriter('runs/exp-2')

In [7]:
def trainer(epoch, model, train_loader, valid_loader, criterion, optimizer, lr_scheduler,
            path, measure):
    maximum = float(-np.inf)
    best_epoch = 0

    result_list = defaultdict()

    for i in range(epoch):

        train_loss, train_gt, train_probs = batch_trainer(
            epoch=i,
            model=model,
            train_loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
        )

        valid_loss, valid_gt, valid_probs = valid_trainer(
            model=model,
            valid_loader=valid_loader,
            criterion=criterion,
        )

        lr_scheduler.step(metrics=valid_loss, epoch=i)

        train_result = get_pedestrian_metrics(train_gt, train_probs)
        valid_result = get_pedestrian_metrics(valid_gt, valid_probs)

        # tensorboard added
        # writer.add_scalar(tag, function, iteration)
        writer_step = i

        writer.add_scalar('Train Loss', train_loss, writer_step)

        writer.add_scalar('Train Accuracy', train_result.instance_acc, writer_step)
        writer.add_scalar('Train Precision', train_result.instance_prec, writer_step)
        writer.add_scalar('Train Recall', train_result.instance_recall, writer_step)
        writer.add_scalar('Train F1', train_result.instance_f1, writer_step)

        writer.add_scalar('Train Mean Accuracy', train_result.ma, writer_step)
        writer.add_scalar('Train Pos Recall', np.mean(train_result.label_pos_recall), writer_step)
        writer.add_scalar('Train Neg Recall', np.mean(train_result.label_neg_recall), writer_step)


        writer.add_scalar('Valid Loss', valid_loss, writer_step)

        writer.add_scalar('Valid Accuracy', valid_result.instance_acc, writer_step)
        writer.add_scalar('Valid Precision', valid_result.instance_prec, writer_step)
        writer.add_scalar('Valid Recall', valid_result.instance_recall, writer_step)
        writer.add_scalar('Valid F1', valid_result.instance_f1, writer_step)

        writer.add_scalar('Valid Mean Accuracy', valid_result.ma, writer_step)
        writer.add_scalar('Valid Pos Recall', np.mean(valid_result.label_pos_recall), writer_step)
        writer.add_scalar('Valid Neg Recall', np.mean(valid_result.label_neg_recall), writer_step)

        print(f'Evaluation on test set, \n',
              'ma: {:.4f},  pos_recall: {:.4f} , neg_recall: {:.4f} \n'.format(
                  valid_result.ma, np.mean(valid_result.label_pos_recall), np.mean(valid_result.label_neg_recall)),
              'Acc: {:.4f}, Prec: {:.4f}, Rec: {:.4f}, F1: {:.4f}'.format(
                  valid_result.instance_acc, valid_result.instance_prec, valid_result.instance_recall,
                  valid_result.instance_f1))

        print(f'{time_str()}')
        print('-' * 60)

        # We only allow "accuracy" or "f1"
        assert((measure.lower()=="accuracy") or (measure.lower()=="f1"))
        if measure == 'accuracy':
            cur_metric = valid_result.ma
        elif measure == 'f1':
            cur_metric = valid_result.instance_f1

        if cur_metric > maximum:
            maximum = cur_metric
            best_epoch = i
            save_ckpt(model, path, i, maximum)

        result_list[i] = [train_result, valid_result]

    writer.close()

    torch.save(result_list, os.path.join(os.path.dirname(path), 'metric_log.pkl'))

    return maximum, best_epoch

In [8]:
class args_config():
    def __init__(self):
        self.dataset="RAP"
        self.debug='store_false'
        self.batchsize=64
        self.train_epoch=30
        self.height=256
        self.width=192
        self.lr_ft=0.01
        self.lr_new=0.1
        self.classifier='base'
        self.momentum=0.9
        self.weight_decay=5e-4
        self.train_split="trainval" #choices=['train' 'trainval']
        self.valid_split="test" #choices=['test' 'valid']
        self.device=0
        self.redirector='store_false'
        self.use_bn='store_false'
        self.train_transform={"Order":["Resize", "Pad", "RandomErase", "RandomCrop", "RandomAffine",\
                                       "RandomHorizontalFlip", "ToTensor", "Normalize", "Mosiac"],
                              "Resize":{"size":(self.height,self.width)},
                              "Pad":{"padding":10},
                              "RandomCrop":{"size":(self.height,self.width)},
                              "RandomAffine":{"degrees":(-5,5), "translate":(0.05,0.05), "scale":(0.99,1.01), "shear":(-5,5)},
                              "RandomHorizontalFlip":{},
                              "RandomErase":{"Wr":(0.05, 0.3), "Hr":(0.05, 0.3)},
                              "Mosaic":{"Wj":0.2, "Hj":0.2, "Prob":0.05},
                              "Normalize":{"mean":[0.485, 0.456, 0.406], "std":[0.229, 0.224, 0.225]}
                             }
        self.valid_transform={"Order":["Resize", "ToTensor", "Normalize"],
                              "Resize":{"size":(self.height,self.width)},
                              "Normalize":{"mean":[0.485, 0.456, 0.406], "std":[0.229, 0.224, 0.225]}
                             }

In [9]:
args = args_config()

In [10]:
# args.dataset = "PA100k"
args.dataset = "PETA"

In [11]:
visenv_name = args.dataset
exp_dir = os.path.join('exp_result', args.dataset)
model_dir, log_dir = get_model_log_path(exp_dir, visenv_name)

# Added for logging purposes
user = getpass.getuser() 
fixed_time_str = time_str()
stdout_file = os.path.join(log_dir, "_".join(['stdout', user, f'{fixed_time_str}.txt']) )
save_model_path = os.path.join(model_dir,  "_".join(['ckpt_max', user, f'{fixed_time_str}.pth']) )
trackitems_dir = os.path.join(log_dir, "_".join(['trackitems', user, f'{fixed_time_str}.txt']) )


if args.redirector:
    print('redirector stdout')
    ReDirectSTD(stdout_file, 'stdout', False)

pprint.pprint(OrderedDict(args.__dict__))

print('-' * 60)
print(f'use GPU{args.device} for training')
print(f'train set: {args.dataset} {args.train_split}, test set: {args.valid_split}')

#train_tsfm, valid_tsfm = get_transform(args)
#print(train_tsfm)

# This is changed
train_set = AttrDataset_new(args=args, split=args.train_split, transformation_dict=args.train_transform)

train_loader = DataLoader(
    dataset=train_set,
    batch_size=args.batchsize,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

# This is changed
valid_set = AttrDataset_new(args=args, split=args.valid_split, transformation_dict=args.valid_transform)

valid_loader = DataLoader(
    dataset=valid_set,
    batch_size=args.batchsize,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)

print(f'{args.train_split} set: {len(train_loader.dataset)}, '
      f'{args.valid_split} set: {len(valid_loader.dataset)}, '
      f'attr_num : {train_set.attr_num}')

labels = train_set.label
sample_weight = labels.mean(0)

backbone = resnet50()
classifier = BaseClassifier(nattr=train_set.attr_num)
model = FeatClassifier(backbone, classifier)

if torch.cuda.is_available():
    model = torch.nn.DataParallel(model).cuda()

criterion = CEL_Sigmoid(sample_weight)

if torch.cuda.is_available():
    param_groups = [{'params': model.module.finetune_params(), 'lr': args.lr_ft},
                    {'params': model.module.fresh_params(), 'lr': args.lr_new}]
else:
    param_groups = [{'params': model.finetune_params(), 'lr': args.lr_ft},
                    {'params': model.fresh_params(), 'lr': args.lr_new}]
optimizer = torch.optim.SGD(param_groups, momentum=args.momentum, weight_decay=args.weight_decay, nesterov=False)
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=4)

# Added for logging purposes
with open(trackitems_dir, "a") as f:
    code, line_no = inspect.getsourcelines(get_transform)
    for line in code:
        f.write(str(line))
    f.write(str("\n\n"))

    f.write(str(args.__dict__))
    f.write(str("\n\n"))

    f.write(str(lr_scheduler.__dict__))
    f.write(str("\n\n"))

    model_str = str(model).lower()
    have_dropout = 'dropout' in model_str
    f.write('dropout: %s' %(have_dropout))
    f.write(str("\n\n"))

    have_leaky_relu = 'leaky_relu' in model_str
    f.write('leaky_relu: %s' %(have_leaky_relu))
    f.write(str("\n\n"))

redirector stdout
OrderedDict([('dataset', 'PETA'),
             ('debug', 'store_false'),
             ('batchsize', 64),
             ('train_epoch', 30),
             ('height', 256),
             ('width', 192),
             ('lr_ft', 0.01),
             ('lr_new', 0.1),
             ('classifier', 'base'),
             ('momentum', 0.9),
             ('weight_decay', 0.0005),
             ('train_split', 'trainval'),
             ('valid_split', 'test'),
             ('device', 0),
             ('redirector', 'store_false'),
             ('use_bn', 'store_false'),
             ('train_transform',
              {'Mosaic': {'Hj': 0.2, 'Prob': 0.05, 'Wj': 0.2},
               'Normalize': {'mean': [0.485, 0.456, 0.406],
                             'std': [0.229, 0.224, 0.225]},
               'Order': ['Resize',
                         'Pad',
                         'RandomErase',
                         'RandomCrop',
                         'RandomAffine',
                     

2020-06-26_07-33-54, Step 99/179 in Ep 6, 0.21s  train_loss:20.0282
2020-06-26_07-33-58, Step 119/179 in Ep 6, 0.21s  train_loss:19.3027
2020-06-26_07-34-03, Step 139/179 in Ep 6, 0.21s  train_loss:22.1589
2020-06-26_07-34-07, Step 159/179 in Ep 6, 0.21s  train_loss:20.3984
2020-06-26_07-34-11, Step 178/179 in Ep 6, 0.06s  train_loss:30.0129
Epoch 6, LR 0.1, Train_Time 38.14s, Loss: 19.8216
Evaluation on test set, 
 ma: 0.7751,  pos_recall: 0.5917 , neg_recall: 0.9586 
 Acc: 0.7363, Prec: 0.8298, Rec: 0.8340, F1: 0.8298
2020-06-26_07-34-19
------------------------------------------------------------
2020-06-26_07-34-25, Step 19/179 in Ep 7, 0.21s  train_loss:18.0113
2020-06-26_07-34-29, Step 39/179 in Ep 7, 0.21s  train_loss:15.1647
2020-06-26_07-34-34, Step 59/179 in Ep 7, 0.21s  train_loss:17.4299
2020-06-26_07-34-38, Step 79/179 in Ep 7, 0.21s  train_loss:18.7968
2020-06-26_07-34-42, Step 99/179 in Ep 7, 0.21s  train_loss:18.7372
2020-06-26_07-34-46, Step 119/179 in Ep 7, 0.21s  tra

2020-06-26_07-41-16, Step 178/179 in Ep 15, 0.06s  train_loss:23.5943
Epoch 15, LR 0.010000000000000002, Train_Time 37.88s, Loss: 9.6337
Evaluation on test set, 
 ma: 0.8127,  pos_recall: 0.6551 , neg_recall: 0.9703 
 Acc: 0.7989, Prec: 0.8755, Rec: 0.8725, F1: 0.8726
2020-06-26_07-41-24
------------------------------------------------------------
2020-06-26_07-41-31, Step 19/179 in Ep 16, 0.21s  train_loss:10.3967
2020-06-26_07-41-35, Step 39/179 in Ep 16, 0.21s  train_loss:8.9182
2020-06-26_07-41-39, Step 59/179 in Ep 16, 0.21s  train_loss:8.9864
2020-06-26_07-41-43, Step 79/179 in Ep 16, 0.21s  train_loss:9.0063
2020-06-26_07-41-48, Step 99/179 in Ep 16, 0.21s  train_loss:9.2741
2020-06-26_07-41-52, Step 119/179 in Ep 16, 0.21s  train_loss:9.6947
2020-06-26_07-41-56, Step 139/179 in Ep 16, 0.21s  train_loss:9.6399
2020-06-26_07-42-00, Step 159/179 in Ep 16, 0.21s  train_loss:7.5959
2020-06-26_07-42-04, Step 178/179 in Ep 16, 0.07s  train_loss:33.1007
Epoch 16, LR 0.01000000000000000

Evaluation on test set, 
 ma: 0.8121,  pos_recall: 0.6543 , neg_recall: 0.9700 
 Acc: 0.8020, Prec: 0.8764, Rec: 0.8756, F1: 0.8746
2020-06-26_07-48-26
------------------------------------------------------------
2020-06-26_07-48-32, Step 19/179 in Ep 25, 0.21s  train_loss:7.8659
2020-06-26_07-48-37, Step 39/179 in Ep 25, 0.21s  train_loss:6.4931
2020-06-26_07-48-41, Step 59/179 in Ep 25, 0.21s  train_loss:6.1461
2020-06-26_07-48-45, Step 79/179 in Ep 25, 0.21s  train_loss:8.1687
2020-06-26_07-48-49, Step 99/179 in Ep 25, 0.21s  train_loss:6.7704
2020-06-26_07-48-53, Step 119/179 in Ep 25, 0.21s  train_loss:6.8386
2020-06-26_07-48-58, Step 139/179 in Ep 25, 0.22s  train_loss:8.4029
2020-06-26_07-49-02, Step 159/179 in Ep 25, 0.21s  train_loss:6.5708
2020-06-26_07-49-06, Step 178/179 in Ep 25, 0.06s  train_loss:35.5921
Epoch 25, LR 0.00010000000000000003, Train_Time 38.25s, Loss: 7.1847
Evaluation on test set, 
 ma: 0.8130,  pos_recall: 0.6558 , neg_recall: 0.9702 
 Acc: 0.8005, Prec: 0

In [12]:
best_metric, epoch = trainer(epoch=args.train_epoch,
                             model=model,
                             train_loader=train_loader,
                             valid_loader=valid_loader,
                             criterion=criterion,
                             optimizer=optimizer,
                             lr_scheduler=lr_scheduler,
                             path=save_model_path,
                             measure='f1')

print(f'{visenv_name},  best_metrc : {best_metric} in epoch{epoch}')

# Added for logging purposes
with open(trackitems_dir, "a") as f:
    f.write(f'{visenv_name},  best_metrc : {best_metric} in epoch{epoch}')
    f.write("\n\n")

100%|██████████| 119/119 [00:07<00:00, 15.04it/s]
